# Task 2: Run Deutsch Algorithm on IonQ Simulator

*In this task, you are already given all the code you’ll need! You need to set up the Azure Quantum connection information and use it to run this code on the IonQ simulator target.*

*To submit this task, update this notebook with the saved results of code cells execution and commit it in your challenge repository.*

*This notebook is part IV of the [Exploring Deutsch-Jozsa Algorithm tutorial](https://github.com/microsoft/QuantumKatas/blob/main/tutorials/ExploringDeutschJozsaAlgorithm/AQ/DeutschJozsaAlgorithmTutorial_P4.ipynb) in the Quantum Katas.*

## Define operations

To start with, we'll define the operations that implement the oracles and the algorithms again; Q# notebooks can use operations defined in them or in .qs files in the same folder, but not in other notebooks.

In [17]:
// Phase oracles we'll use to demonstrate Deutsch algorithm
operation PhaseOracle_Zero (x : Qubit) : Unit {
    // Do nothing...
}

operation PhaseOracle_X (x : Qubit) : Unit {
    Z(x);
}

operation DeutschAlgorithm (oracle : (Qubit => Unit)) : Result {
    use x = Qubit();
    H(x);
    oracle(x);
    H(x);
    return M(x);
}

PhaseOracle_Zero, PhaseOracle_X, DeutschAlgorithm

Note that we've modified the signature of the operation that implements Deutsch algorithm to return `Result` instead of `Bool`; `Zero` indicates a constant function and `One` - a balanced function. We'll see why we needed this change once we get to running the algorithm on hardware!

After this, we need to define the wrapper operation that will call the Deutsch algorithm with the right oracle parameter depending on a string parameter.
Q# notebooks allow to pass parameters of different types to operations executed via magics, but they cannot pass callables as parameters.

In [18]:
operation DeutschAlgorithmWrapper (oracleFunction : String) : Result {
    Message($"Running Deutsch algorithm for f(x) = {oracleFunction}");
    if oracleFunction == "0" {
        return DeutschAlgorithm(PhaseOracle_Zero);
    } elif oracleFunction == "x" {
        return DeutschAlgorithm(PhaseOracle_X);
    } else {
        fail $"Unknown function f(x) = {oracleFunction}";
    }
}

DeutschAlgorithmWrapper

Let's run the algorithm on a simulator like we did before, to make sure we didn't introduce any unexpected behaviors (think of it as unit tests for your code!)

In [19]:
%simulate DeutschAlgorithmWrapper oracleFunction=0

Running Deutsch algorithm for f(x) = 0


Zero

In [20]:
%simulate DeutschAlgorithmWrapper oracleFunction=x

Running Deutsch algorithm for f(x) = x


One

## Connect to Azure Quantum

You need to substitute `<subscriptionId>`, `<resourceGroupId>`, `<workspaceId>`, and `<location>` in the connection string below with the parameter values from your own quantum workspace.
You can learn how to create an Azure Quantum workspace in [this Microsoft Learn module](https://docs.microsoft.com/learn/modules/get-started-azure-quantum/3-create-quantum-workspace).
(Executing this cell will open a new tab with a login prompt; return to the notebook once you log in.)

In [30]:
%azure.connect "/subscriptions/b0c3bbe3-7123-4ba4-9aa2-57d9845f4a1d/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/qchack-2022-b" location="eastus"

Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace qchack-2022-b in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,45
ionq.simulator,Available,2
quantinuum.hqs-lt-s1,Unavailable,0
quantinuum.hqs-lt-s1-apival,Available,0
quantinuum.hqs-lt-s2,Unavailable,0
quantinuum.hqs-lt-s2-apival,Available,0
quantinuum.hqs-lt-s1-sim,Available,68
quantinuum.hqs-lt-s2-sim,Available,72
quantinuum.hqs-lt,Unavailable,0


The `%azure.connect` magic outputs the list of targets available in your subscription. 
If you added IonQ as the provider to your workspace, you'll get `ionq.simulator` (the classical simulator of a quantum device) and `ionq.qpu` (the actual quantum device).
Let's submit a job for the simulator first; we'll need to choose the target using the `%azure.target` magic command.

In [31]:
%azure.target ionq.simulator

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.simulator,Available,2


Now, you can submit a job to the current target using the `%azure.submit` magic command which takes the same parameters as the `%simulate` magic command used earlier.

When you run this cell, you'll get a job id in return. You'll use this id for looking up the job status, either via the `%azure.status` command or via Azure Portal, and for retrieving job results.

In [32]:
%azure.submit DeutschAlgorithmWrapper oracleFunction=0

Submitting DeutschAlgorithmWrapper to target ionq.simulator...
Job successfully submitted for 500 shots.
   Job name: DeutschAlgorithmWrapper
   Job ID: 407c0041-98c2-4cd0-bc40-d477eb69c8bb


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time,Cost Estimate
DeutschAlgorithmWrapper,407c0041-98c2-4cd0-bc40-d477eb69c8bb,Waiting,ionq.simulator,04/09/2022 19:42:25 +00:00,,,


Running `%azure.status` without any arguments will fetch the status of the latest job executed in this notebook; if you need to look up an earlier job, you can use its id as a parameter.

In [37]:
%azure.status

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time,Cost Estimate
DeutschAlgorithmWrapper,407c0041-98c2-4cd0-bc40-d477eb69c8bb,Succeeded,ionq.simulator,04/09/2022 19:42:25 +00:00,04/09/2022 19:43:00 +00:00,04/09/2022 19:43:00 +00:00,$0.00


Similarly, running `%azure.output` without any arguments will fetch the results of the latest job executed in this notebook; if you need to look up an earlier job, you can use its id as a parameter.

In [38]:
%azure.output

Result,Frequency,Histogram
0,1,


The format of job execution results differs from the results produced by the local simulator; instead of a single run result, you get a histogram of results of multiple job runs ("shots").
In this case, though, the job we run produces a deterministic result, and in the absence of noise the histogram consists of a single entry.
If you run the same job on quantum hardware without error correction, or on a noisy simulator, you'll get a distribution of multiple results, and will need to analyze the data to figure out the right answer.